In [70]:
import pandas as pd
import numpy as np

df = pd.read_csv('xor-weight.csv', sep=',')
df['weight'] = list(zip(df.to, df.weight))
df.drop(['to'], axis=1, inplace=True)
df
# group_data = df.groupby(['layerFrom', 'from', 'layerTo'])
# print(group_data)
# ans = group_data.aggregate({'weight': lambda x: list(np.unique(np.concatenate(x.values)))})
# print(ans)

,layerFrom,from,layerTo,weight
0,input,x0,hidden1,"(h1, -10)"
1,input,x0,hidden1,"(h2, -30)"
2,input,x1,hidden1,"(h1, 20)"
3,input,x1,hidden1,"(h2, -20)"
4,input,x2,hidden1,"(h2, 20)"
5,input,x2,hidden1,"(h2, -20)"
6,hidden1,x0,output,"(y, -30)"
7,hidden1,h1,output,"(y, 20)"
8,hidden1,h2,output,"(y, 20)"
